# NDVI

This document shows the computation of NDVI scores for each of the enclosed tesselations (ETs) in GB.

In [1]:
! echo "Run this notebook using version $GDS_ENV_VERSION of the gds_env"
#SERVER_IP = open("../../SERVER_IP").read().strip("\n")
SERVER_IP = "0.0.0.0"

Run this notebook using version  of the gds_env


## Set up

Since we will run some computations on a Dask cluster, let's set it up first:

In [2]:
! cat ../../worker-spec.yml

cat: ../../worker-spec.yml: No such file or directory


In [1]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client
import dask.array as da

# Set up cluster
cluster = KubeCluster.from_yaml('../../worker-spec.yml')
# Provision with up to 30 pods
cluster.scale(50)
# Connect Dask to the cluster
client = Client(cluster)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: tcp://138.253.73.24:37297
distributed.scheduler - INFO -   dashboard at:                     :8787
distributed.scheduler - INFO - Receive client connection: Client-106cf4b4-3296-11eb-8594-80e82cd20b5e
distributed.core - INFO - Starting established connection


In [2]:
# Local run (alternative to cluster)
from dask.distributed import Client
import dask.array as da

client = Client()

Bring on other libs we'll use:

In [3]:
import sys
sys.path.insert(0, "../")
import utils
import os
import fsspec
import pandas
import geopandas
import rioxarray, xarray
from dask import dataframe as dd
from numpy import percentile

And paths to the two datasets we'll use:

- The full mosaic is stored as a folder of COGs served over HTTP. First let's grab the URL for the mosaic:

In [4]:
mosaic_url = f"http://{SERVER_IP}:8000/ghs_composite_s2/GHS-composite-S2.vrt"

- The enclosed tessellation:

In [23]:
tess_path = f"http://{SERVER_IP}:8000/tessellation/"

- And the list of chunks:

In [24]:
chunks = [f"{tess_path}tess_{i}.pq" for i in range(103)]

## Connecting to the GHS Mosaic

We inspect the details of the mosaic to select the chunk:

In [21]:
! rio info $mosaic_url | python -m json.tool

{
    "blockxsize": 128,
    "blockysize": 128,
    "bounds": [
        -222823.73719089525,
        -213574.25107683009,
        996789.2497132053,
        1612237.380579703
    ],
    "colorinterp": [
        "gray",
        "undefined",
        "undefined",
        "undefined"
    ],
    "count": 4,
    "crs": "EPSG:27700",
    "descriptions": [
        null,
        null,
        null,
        null
    ],
    "driver": "VRT",
    "dtype": "uint16",
    "height": 182437,
    "indexes": [
        1,
        2,
        3,
        4
    ],
    "lnglat": [
        -2.2113098420427835,
        56.186432587438965
    ],
    "mask_flags": [
        [
            "nodata"
        ],
        [
            "nodata"
        ],
        [
            "nodata"
        ],
        [
            "nodata"
        ]
    ],
    "nodata": 0.0,
    "res": [
        10.007902079383749,
        10.007902079383749
    ],
    "shape": [
        182437,
        121865
    ],
    "tiled": true,
    "transform"

Since it's tiled on 128 by 128 pixels, we pick a chunk size that is ten times larger:

In [5]:
r = rioxarray.open_rasterio(mosaic_url,
                            chunks={"x": 1280, "y": 1280}
                           )
r

,Array,Chunk
Bytes,177.86 GB,13.11 MB
Shape,"(4, 182437, 121865)","(4, 1280, 1280)"
Count,13729 Tasks,13728 Chunks
Type,uint16,numpy.ndarray


This will make each chunk in `r` read ten tiles at a time.

## Enclosed Tesselation (ET) cells

The ET cells are stored as parquet files by geographic chunks. Let's read a random one (`#6`) for this illustration:

In [27]:
db_path = tess_path + "tess_6.pq"
with fsspec.open(db_path) as file:
    tst = geopandas.read_parquet(file, 
                                columns=["hindex", "tessellation"]
                               )
tst.head()

,hindex,tessellation
0,c006e658591t0000,"POLYGON Z ((356038.093 215849.818 0.000, 35603..."
1,c006e658591t0105,"MULTIPOLYGON Z (((355777.262 216080.707 0.000,..."
2,c006e658591t0106,"POLYGON Z ((355789.951 215942.223 0.000, 35577..."
3,c006e658591t0107,"POLYGON Z ((355804.850 215994.579 0.000, 35578..."
4,c006e658591t0108,"POLYGON Z ((355791.991 216043.449 0.000, 35578..."


In [28]:
tst_sub = tst.cx[315891.95:330000, 213727.69:250000]
tst_sub.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2049 entries, 15006 to 258237
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   hindex        2049 non-null   object  
 1   tessellation  2049 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 48.0+ KB


## Explore `DataArray` size by chunk

In sparsely populated areas, ET cells will tend to be larger and, thus translate into larger sizes of the imagery required to cover it. Let's explore the extent to which this is an issue.

The `chunk_dimension` method below completes the following tasks for a given chunk:

1. Collect bounding boxes for each chunk
1. Connect to that section of the mosaic
1. Extract dimensions

In [6]:
def format_bytes(size):
    """
    Convert bytes to human-readable units
    
    Taken from: https://stackoverflow.com/a/49361727
    ...
    
    Arguments
    ---------
    size : int
          File size
    
    Returns
    -------
    fsize : int
           Formatted file size
    unit : str
          Unit in which size is expressed
    """
    # 2**10 = 1024
    power = 2 ** 10
    n = 0
    power_labels = {0: "", 1: "kilo", 2: "mega", 3: "giga", 4: "tera"}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n] + "bytes"

def chunk_dimension(chunk_path):
    if type(chunk_path) is not str:
        chunk_path = chunk_path["paths"]
    with fsspec.open(chunk_path) as p:
        chunk = geopandas.read_parquet(p, columns=["hindex", "tessellation"])
    section = r.rio.clip_box(*chunk.total_bounds)
    x_range = section.coords["x"].max() - section.coords["x"].min()
    y_range = section.coords["y"].max() - section.coords["y"].min()
    area = x_range * y_range / 1e6
    size, unit = format_bytes(section.nbytes)
    p = chunk_path.split("/")[-1]
    out = pandas.Series([p, float(area), section.nbytes, size, unit], 
                        index=["cID", "area_sqkm", "size", "hsize", "unit"]
                       )
    return out

And we can run this distributed across the Dask client:

In [144]:
%%time

chunks_dd = dd.from_pandas(pandas.DataFrame({"paths": chunks}), 
                           npartitions=8
                          )
sizes = chunks_dd.map_partitions(lambda df: df.apply(chunk_dimension, axis=1),
                                 meta=[("cID", "str"),
                                       ("area_sqkm", "f8"),
                                       ("size", "f8"),
                                       ("hsize", "f8"),
                                       ("unit", "str")
                                      ]
                                ).compute()

CPU times: user 6.49 s, sys: 225 ms, total: 6.71 s
Wall time: 39.1 s


,cID,area_sqkm,size,hsize,unit
0,tess_0.pq,815.881906,65213280,62.192230,megabytes
1,tess_1.pq,1557.383900,124457600,118.692017,megabytes
2,tess_2.pq,1010.102091,80731976,76.992012,megabytes
3,tess_3.pq,1448.551503,115762080,110.399323,megabytes
4,tess_4.pq,4745.447901,379148176,361.583878,megabytes
...,...,...,...,...,...
98,tess_98.pq,1580.891808,126337344,120.484680,megabytes
99,tess_99.pq,5186.599478,414389008,395.192154,megabytes
100,tess_100.pq,16602.280239,1326292344,1.235206,gigabytes
101,tess_101.pq,1926.794331,153971600,146.838760,megabytes


From the `sizes` table we can see there's a few chunks that cover a large swath of land, but most are very manageable otherwise. Hence, what we will do is run in parallel all the smaller chunks and then sequentially the few larger ones. In particular, we will leave aside the top three:

In [145]:
sizes.sort_values("size", ascending=False).head(10)

,cID,area_sqkm,size,hsize,unit
5,tess_5.pq,98852.771104,7896278496,7.353982,gigabytes
87,tess_87.pq,98381.295659,7858582248,7.318875,gigabytes
97,tess_97.pq,33881.959783,2706574856,2.520694,gigabytes
83,tess_83.pq,23445.222729,1872902528,1.744276,gigabytes
89,tess_89.pq,18351.235343,1466001600,1.365320,gigabytes
48,tess_48.pq,17581.976333,1404555840,1.308095,gigabytes
38,tess_38.pq,16949.749936,1354048752,1.261056,gigabytes
100,tess_100.pq,16602.280239,1326292344,1.235206,gigabytes
71,tess_71.pq,15812.966472,1263241712,1.176486,gigabytes
85,tess_85.pq,11608.782981,927409416,884.446541,megabytes


In [146]:
to_run_seq = [5, 87, 97]

## Distribute computation by ET cell

To calculate NDVI for each ET cell, we create two methods:

In [19]:
def geom2ndvi(row, ndvi, geom="geometry"):
    val = ndvi.rio.clip([row[geom]])\
              .mean()\
              .values\
              .tolist()
    return val

def ndvi_from_chunk(db_path,
                    mosaic_path=mosaic_url, 
                    geom="tessellation"
                   ):
    if type(db_path) is pandas.Series:
        db_path = db_path.iloc[0]
    with fsspec.open(db_path) as file:
        db = geopandas.read_parquet(file, 
                                    columns=["hindex", "tessellation"]
                                   ).set_index("hindex").head() # <-- head only!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    r = rioxarray.open_rasterio(mosaic_path,
                                chunks={
                                    "x": 640, 
                                    "y": 640
                                }
                               )
    ndvi = (r.sel(band=4) - r.sel(band=1)) / \
           (r.sel(band=4) + r.sel(band=1))
    ndvi_vals = ndvi.rio.clip_box(*db.total_bounds).load()
    rower = lambda row: geom2ndvi(row, ndvi_vals, geom=geom)
    return db.apply(rower, axis=1)

The first, `geom2ndvi` takes a single row with the geometry of the cell, and a `DataArray` and returns the NDVI for that cell.

This method can be applied sequentially to an entire table from the URL of the mosaic, which is what we describe in `ndvi_from_chunk`.

**NOTE** For this to work effectively, the extent of `db.total_bounds` needs to fit comfortably in memory

To run the above distributedly, we will set up a Dask DataFrame with all the URLs of the file names:

In [41]:
get_chunk_no = lambda c: int(c.split("tess_")[1].strip(".pq"))
chunk_names = dd.from_pandas(pandas.DataFrame({"file": chunks,
                                               "cID": list(map(get_chunk_no, chunks))
                                              }),
                                              chunksize=1
                                             )
chunk_names

In [ ]:
chunk_names.compute()

Now we can map the computation of each chunk across the cluster:

In [ ]:
%%time
ndvis = chunk_names["file"].map_partitions(ndvi_from_chunk, 
                                   meta=(None, 'f8')
                                  ).compute()

In [28]:
ndvis

hindex
c000e109777t0000    0.196805
c000e109777t0001    0.165362
c000e109777t0002    0.195056
c000e109777t0003    0.306654
c000e109777t0004    0.216186
c001e242798t0000    0.447537
c001e242798t0035    0.345079
c001e242798t0026    0.228298
c001e242798t0032    0.267280
c001e242798t0033    0.243486
c002e355065t0000    8.415008
c002e355065t0001    0.333604
c002e355065t0002    1.860787
c002e687334t0000    0.532744
c002e687334t0001    0.358899
c003e503705t0000    0.386689
c003e503705t0024    0.513999
c003e503705t0023    0.466799
c003e503705t0022    0.554348
c003e503705t0021    0.422845
dtype: float64

## National NDVI resampling

In this section we resample the national NDVI to explore rasterisation in Dask and be able to create an image of the distribution of NDVI at the national scale.

We can express the calculation of the NDVI index, although no computation will take place thanks to `xarray`/Dask's lazy evaluation:

In [6]:
ndvi = (r.sel(band=4) - r.sel(band=1)) / (r.sel(band=4) + r.sel(band=1))

With `datashader`, we can resample the entire mosaic into a much smaller `DataArray` that will fit comfortably in memory and that we can then plot:

In [8]:
import datashader as ds

cvs = ds.Canvas(plot_width=1212, plot_height=1824)
agg = cvs.raster(ndvi)
agg

The real computation happens below:

In [9]:
%%time
ndvi_agg = agg.compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('concatenate-finalize-from-value-resample_2d-getitem-06ea23e4deba23af57f89b9974ecd458', 0, 0)", <Worker 'tcp://127.0.0.1:42509', name: 2, memory: 0, processing: 1139>)

In [148]:
ndvi

,Array,Chunk
Bytes,177.86 GB,13.11 MB
Shape,"(182437, 121865)","(1280, 1280)"
Count,82369 Tasks,13728 Chunks
Type,float64,numpy.ndarray


---

## [DEPRECATED] Alternative using `geocube`

The alternative involves [`geocube`'s zonal stats](https://corteva.github.io/geocube/stable/examples/zonal_statistics.html) and `make_geocube`. In this approach, we first rasterize our ET cells in a grid aligned with the mosaic, then calculate the NDVI. At present, this approach is discarded because the resolution of the mosaic (10m) makes it too coarse to obtain an NDVI for each cell.

In [57]:
from geocube.api.core import make_geocube

We use the same set of ET cells:

In [ ]:
url = f"http://{SERVER_IP}:8000/spatial_signatures/tessellation/tess_6.pq"
tst = geopandas.read_parquet(url)


In [117]:
tst_sub = tst.cx[315891.95:330000, 213727.69:250000]
tst_sub.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2049 entries, 15006 to 258237
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   uID          2049 non-null   int64   
 1   geometry     2049 non-null   geometry
 2   enclosureID  2049 non-null   int64   
dtypes: geometry(1), int64(2)
memory usage: 64.0 KB


Before rasterization, we need to load the segment of the mosaic that overlaps (note no bits are streamed to memory, all lazy evaluation):

In [118]:
ndvi_segment = ndvi.rio.clip_box(*tst_sub.total_bounds)

We need to rasterize the features:

In [119]:
%%time
out_grid = make_geocube(
    vector_data = tst_sub,
    measurements=["uID"],
    like=ndvi_segment
)

CPU times: user 318 ms, sys: 5.81 ms, total: 324 ms
Wall time: 323 ms


This creates a `DataSet` object with a rasterised version of the tesselations in `tst`. Now we append the NDVI:

In [120]:
out_grid["ndvi"] = ndvi_segment

And with both aligned, we can group by each `uID` and calculate average NDVI:

In [121]:
%%time
g = out_grid.drop("spatial_ref")\
            .groupby(out_grid["uID"])

CPU times: user 598 ms, sys: 2.23 ms, total: 601 ms
Wall time: 598 ms


And we can get the average easily:

In [127]:
%%time
ndvi_mean = g.mean()

CPU times: user 6.21 s, sys: 2.89 ms, total: 6.22 s
Wall time: 6.21 s


In [132]:
mn = ndvi_mean.to_dataframe()[["ndvi"]]

/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [149]:
geom2ndvi(tst_sub.query("uID == 6712717").iloc[0], ndvi)

/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


0.6968843539763994

In [153]:
%time out = tst_sub.head().apply(lambda r: geom2ndvi(r, ndvi), axis=1)

/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


CPU times: user 1.6 s, sys: 14.6 ms, total: 1.62 s
Wall time: 1.6 s


/opt/conda/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [144]:
tst_sub.query("uID == 6712717")

,uID,geometry,enclosureID
22988,6712717,"POLYGON Z ((329700.369 234529.467 0.000, 32945...",658102


In [141]:
mn.head()

,ndvi
uID,
6712717.0,NaN
6712718.0,0.682241
6712719.0,0.675091
6712720.0,NaN
6712721.0,0.537544
